In [1]:
# !pip install chromedriver_autoinstaller undetected_chromedriver

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for undetected_chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47184 sha256=a89449c51685d492043315b53bf61aeba51c367b549b08f78b339b0a9d69717f
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\d1\9c\b4\ccdca75ca356899a9b65190f048a03058f1fe23602c642b88e
Successfully built undetected_chromedriver



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import os
import json
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import random
import chromedriver_autoinstaller


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def find_first_clickable(driver, locators, timeout=5):
    """
    locators: 리스트 of (By, locator_string) 튜플
    """
    for by, loc in locators:
        try:
            # 1) 요소가 DOM 에 올라오고, 화면에 보여질 때까지
            el = WebDriverWait(driver, timeout).until(
                EC.visibility_of_element_located((by, loc))
            )
            # 2) 스크롤이 해당 요소를 가운데로 오도록
            driver.execute_script(
                "arguments[0].scrollIntoView({block:'center'});", el
            )
            # 3) 클릭 가능 상태가 될 때까지
            WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((by, loc))
            )
            return el
        except Exception:
            continue
    return None

review_locators = [
    (By.XPATH, '//*[@id="content"]/div[4]/div[3]/div[1]/div/ul/li[2]/a'),
    (By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a'),
    (By.XPATH, '//*[@id="content"]/div[4]/div[2]/div[1]/div/ul/li[2]/a'),
]

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:115.0) Gecko/20100101 Firefox/115.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_0) AppleWebKit/605.1.15 "
    "(KHTML, like Gecko) Version/16.5 Safari/605.1.15",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) "
    "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 8 Pro) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36",
]

def set_random_ua(driver):
    ua = random.choice(USER_AGENTS)
    # Network 도메인 활성화 후 User-Agent 덮어쓰기
    driver.execute_cdp_cmd("Network.enable", {})
    driver.execute_cdp_cmd("Network.setUserAgentOverride", {"userAgent": ua})
    print(f"[UA ROTATE] new UA → {ua}")

# 1) 현재 설치된 Chrome 버전에 맞는 chromedriver를 자동으로 설치
chromedriver_autoinstaller.install()

# 다운로드 경로
download_dir = ".\\download_json"
os.makedirs(download_dir, exist_ok=True)

# Chrome + 스텔스 모드 설정
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-popup-blocking")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 9.0; Win64; x64) '
#                      'AppleWebKit/537.37 (KHTML, like Gecko) Chrome/119.1.0.0 Safari/538.36')
                     

# 드라이버 실행
driver = uc.Chrome(version_main=135, options=options)
set_random_ua(driver)
driver.get('https://shopping.naver.com/gift/home')
time.sleep(random.uniform(1,3))

# 카테고리 클릭
xpath_1 = f'//*[@id="content"]/div/div[1]/ul/li[2]/a'
btn_1 = driver.find_element(By.XPATH, xpath_1)
btn_1.click()
time.sleep(random.uniform(2,3.5))

# 상품 카테고리 클릭
xpath_2 = f'//*[@id="content"]/div/div[2]/div[2]/ul/li[2]/button'
btn_2 = driver.find_element(By.XPATH, xpath_2)
btn_2.click()
time.sleep(random.uniform(2,3.5))


# 유아동 카테고리 클릭
xpath_3 = f'//*[@id="content"]/div/div[2]/div[3]/div[1]/div/div/div/ul/li[11]/button'
btn_3 = driver.find_element(By.XPATH, xpath_3)
btn_3.click()
time.sleep(random.uniform(1,3))
name_txt1 = driver.find_element(By.XPATH, xpath_3).text.split()[0]
# print(name_txt1)

for i in range(1, 2):# 9개
    products_data = []
    
    # 세부 카테고리 클릭
    xpath_4 = f'//*[@id="content"]/div/div[2]/div[3]/div[1]/div/div[2]/div/ul/li[{i+1}]/button'
    btn_4 = driver.find_element(By.XPATH, xpath_4)
    btn_4.click()
    time.sleep(random.uniform(2,3.5))
    name_txt2 = driver.find_element(By.XPATH, xpath_4).text.split()[0]
    # print(name_txt2)
    
    output_path = os.path.join(download_dir, "products_babys_" + name_txt2.replace("/", "") +".json")
    original_window = driver.current_window_handle

    xpath_4 = f'//*[@id="content"]/div/div[2]/div[4]/div[3]/ul'
    # 상품 로드
    while True:
        goods_1 = driver.find_element(By.XPATH, xpath_4)
        goods_list = goods_1.find_elements(By.TAG_NAME, "li")
        count = len(goods_list)
        print(f"현재 로드된 상품 개수: {count}")
        if count >= 80:
            break
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(2,3.5))

    goods_list = goods_list[:80]
    print(len(goods_list))

    # 상품 선택
    for j, item in enumerate(goods_list):
        print(name_txt1, name_txt2, " ", j+1)
        # url = item.find_element(By.XPATH, f'/html/body/div[4]/div/div[2]/div/div[2]/div[4]/div[3]/ul/li[{j+1}]/div/a')
        # url = item.find_element(By.CSS_SELECTOR, 'li > div > a')
        # driver.execute_script("window.open(arguments[0], '_blank');", url)
        # time.sleep(2)if idx % 10 == 1 and idx != 1:
        if j % 10 == 1 and j != 1:
            set_random_ua(driver)
        href  = item.find_element(By.CSS_SELECTOR, 'li > div > a').get_attribute("href")
        driver.switch_to.new_window("tab")
        driver.get(href)
        time.sleep(random.uniform(2,3.5))
        
        new_handle = [h for h in driver.window_handles if h != original_window][0]
        driver.switch_to.window(new_handle)
        time.sleep(random.uniform(2,3.5))

        # 상품명
        name_txt3 = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/div[2]/fieldset/div[2]/div[1]/h3').text
        print(name_txt3)

        # 상품 가격
        price_loc = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/div[2]/fieldset/div[2]/div[2]/div/strong/span[2]')
        price = price_loc.text
        price = price.replace(",", "").strip()
        print(price)
        
        # 상품 브랜드
        try:
            brand_loc = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/div[2]/fieldset/div[1]/div[2]/div/a')
            brand = brand_loc.text
            print(brand)
        except NoSuchElementException:
            print('브랜드 없음')
            brand = None

        # 상품 이미지
        try:
            img_elem = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/div[1]/div[1]/div[1]/img')
            link_save = img_elem.get_attribute('src')
            print(link_save)
        except NoSuchElementException:
            pass
            
        # 상품 옵션
        option_list = []
        try:
            # 옵션창 열기
            option_open = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/div[2]/fieldset/div[7]/div/a')
            option_open.click()
            option_ul = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/div[2]/fieldset/div[7]/div/ul')
            li_elements = option_ul.find_elements(By.TAG_NAME, "li")
            for k in li_elements:
                option_text = k.find_element(By.TAG_NAME, "a").text
                option_list.append(option_text)

        except NoSuchElementException:
            print('옵션 없음')
            pass
        print(option_list)

        # 상품 링크
        current_url = driver.current_url
        print("상품 링크:", driver.current_url)

        # 리뷰내용
        review_list = []
        try:
            # try:
            #     xpath_5 = f'//*[@id="content"]/div[4]/div[3]/div[1]/div/ul/li[2]/a'
            #     btn_5 = driver.find_element(By.XPATH, xpath_5)
            # except:
            #     xpath_5 = f'//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a'
            #     btn_5 = driver.find_element(By.XPATH, xpath_5)
            # btn_5 = find_first(driver, By.XPATH, review_xpaths)
            # print(btn_5)
            # btn_5.click()
            btn_5 = find_first_clickable(driver, review_locators)
            if btn_5:
                driver.execute_script("arguments[0].click();", btn_5)
            else:
                print("리뷰 탭 버튼을 찾지 못했거나 클릭 불가 상태입니다.")
            time.sleep(random.uniform(2,3.5))

            for l in range(1, 101):
                try:
                    xpath_6 = f'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li[{((l - 1) % 20) + 1}]/div/div/div/div[1]/div/div/div[2]/div/span'
                    review = driver.find_element(By.XPATH, xpath_6)
                    review_text = review.text
                    review_list.append(review_text)

                    # 다음 페이지 클릭
                    if l % 20 == 0:
                        try:
                            page_num = (l // 20) + 2
                            btn_6 = driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page_num}]')
                            btn_6.click()
                            time.sleep(2)
                        except NoSuchElementException:
                            pass
                        
                except NoSuchElementException:
                    pass
        except NoSuchElementException:
            pass

        print(review_list)

        products_data.append({
            "brand_name": brand,
            "product_name": name_txt3,
            "category_main": name_txt1,
            "category_sub": name_txt2,
            "options": option_list,
            "product_url": current_url,
            "thumbnail_url": link_save,
            "review_content": review_list,
            "price": price
        })

        driver.close()
        # driver.switch_to.window(original_window)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(products_data, f, ensure_ascii=False, indent=2)
    
# driver.quit()

[UA ROTATE] new UA → Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_0) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15
현재 로드된 상품 개수: 40
현재 로드된 상품 개수: 80
80
유아동 완구/교구   1
[띵이슬라임] 깡졍마켓 수제 슬라임 거품빵 120ml 액체괴물 스트레스 해소 선물
6900
깡졍마켓
https://shop-phinf.pstatic.net/20240624_105/1719222396343bpmmq_JPEG/579986121104311_2007315591.jpg?type=m510
['거품빵', '마시모찌', '만년설', '명이떡집', '묵쫀퐁', '분퐁말', '유채휘핑', '청사과크림요거트', '퐁숭아', '라벤폼', '치즈밥', '쿠생케', '쿠크볼', '라이스바삭바', '레드프룻', '알로에 수딩젤리', '소다샤베트', '콜라슬러쉬', '미국코코아 베이스', '완두앙꼬', '고급수건']
상품 링크: https://shopping.naver.com/gift/products/6208431184?NaPm=ct%3Dmae1bz2i%7Cci%3Dshoppingwindow%7Ctr%3Dgsh%7Chk%3Db6ff7be7cf665699db5fd9f5e6100b0d3e19f961%7Ctrx%3D
['재구매', '일단 슬라임이 한참 유행할때도 굳이? 사야하나? 하면서 그냥 영상만 보던 사람인데 요즘 손을 가만히 두는게 심심하기도 하고 오랜만에 슬라임이 알고리즘에 떠서 막 너무 갖고싶어져서 질려버렸는데요! 일단 너무너무 만족합니다! 띵슬이 젤 유명한 있었네요! 전 완두앙꼬, 수딩젤리, 치즈밥, 거품빵, 청사과 크림 요거트 샀고 서비스 슬라임으로 피자도우가 왔는데요! 슬린이인 제가 찬찬히 리뷰 써보자면!\n일단 완두앙꼬!\n제가 딱 생각한 베이스 슬라임 느낌입니다! 뭔가 다른 슬라임을 많이 만져보질 않아서 비교는 못 하지만 딱 오

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/div[3]/div[2]/fieldset/div[2]/div[1]/h3"}
  (Session info: chrome=136.0.7103.49); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00D5D363+60275]
	GetHandleVerifier [0x00D5D3A4+60340]
	(No symbol) [0x00B906F3]
	(No symbol) [0x00BD8690]
	(No symbol) [0x00BD8A2B]
	(No symbol) [0x00C20EE2]
	(No symbol) [0x00BFD0D4]
	(No symbol) [0x00C1E6EB]
	(No symbol) [0x00BFCE86]
	(No symbol) [0x00BCC623]
	(No symbol) [0x00BCD474]
	GetHandleVerifier [0x00FA8FE3+2467827]
	GetHandleVerifier [0x00FA45E6+2448886]
	GetHandleVerifier [0x00FBF80C+2560028]
	GetHandleVerifier [0x00D73DF5+153093]
	GetHandleVerifier [0x00D7A3BD+179149]
	GetHandleVerifier [0x00D64BB8+91080]
	GetHandleVerifier [0x00D64D60+91504]
	GetHandleVerifier [0x00D4FA10+4640]
	BaseThreadInit

In [ ]:
len(products_data)

In [8]:
len(review_list)

100